In [1]:
import pandas as pd
import os
from datetime import datetime

In [2]:
path = "C:/Users/Hp/OneDrive/Documents/Desktop/10june/weekly_data/"
file_list = os.listdir(path)
df = pd.read_csv(path+file_list[0])
df.head()

,Ticker,Date/Time,Open,High,Low,Close,Volume,Open Interest
0,BANKNIFTY,31-12-2021 09:15:00,35114.4,35225.7,35113.8,35220.0,1,0
1,BANKNIFTY,31-12-2021 09:16:00,35218.8,35285.8,35218.8,35276.9,1,0
2,BANKNIFTY,31-12-2021 09:17:00,35289.3,35320.8,35275.7,35311.9,1,0
3,BANKNIFTY,31-12-2021 09:18:00,35311.6,35333.5,35301.4,35307.9,1,0
4,BANKNIFTY,31-12-2021 09:19:00,35311.2,35329.6,35302.3,35329.6,1,0


In [3]:
def convert(date):
    format = "%d-%m-%Y %H:%M:%S"
    return datetime.strptime(date,format)

In [4]:
def find_day(date):
    return date.strftime('%A')

In [1]:
def get_atm(spot):
    rem = spot%100
    if rem>=50:
        return int(spot/100)*100+100
    return int(spot/100)*100

In [2]:
get_atm(44339)

44300

In [49]:
result = pd.DataFrame()

In [50]:
var = {'status':False,'entry_time':None,  'index_ltp' : None, 'call_strike':None, 'put_strike':None,'entry_ce_ltp':None, 'entry_pe_ltp':None, 'call_sl':None, 'put_sl':None,
               'exit_time_call':None, 'exit_time_put':None, 'exit_ce_ltp':None, 'exit_pe_ltp':None,
               'call_pnl':0, 'put_pnl':0,
               'day_pnl':0, 'cumulative_pnl':0, 'remark':None}


In [51]:
result = pd.DataFrame()

In [52]:
for file in file_list:
    week = pd.read_csv(path+file)
    
    # making the index date/time and changing the type from string to datetime
    dates = []
    for date in week['Date/Time']:
        dates.append(convert(date))
    del(week["Date/Time"])
    week["Date/Time"] = dates
    week.index = week["Date/Time"]
    
    # extracting the data of banknifty 
    bn_index = week[week['Ticker']=='BANKNIFTY']
    
    # finding the unique days of the all date/time given in the week
    date_times = []
    for date,_ in bn_index.iterrows():
        date_times.append(str(date)[:10])
    date_times = pd.Series(date_times).unique()
    unique_day = []
    for el in date_times:
        format = "%Y-%m-%d"
        unique_day.append(datetime.strptime(el,format))
    
    for date in unique_day:
        # list of position which will be taken in a day
        p1 = var.copy()
        p2 = var.copy()
        p3 = var.copy()
        p4 = var.copy()
        
        # start_time, end_time, intraday_time
        start_time = str(date)[:10]+ ' 09:20:00'
        end_time = str(date)[:10]+ ' 15:20:00'
        intraday_time = [str(x) for x  in pd.date_range(start=start_time,end=end_time,freq='min')]
        
        # Traverse over the intraday time defined above
#         print(find_day(date))
        if find_day(date)=='Monday' or find_day(date)=='Friday':
#             print("kfdjkdj")
            for time in intraday_time:
        
                # checking for first position of a day
                if p1['status']==False or time == str(date)[:10]+ ' 09:40:00':
#                     print('check')
                    p1['status'] = True
                    p1['entry_time'] = time
                    
                    # finding the index ltp
                    row1 = bn_index.loc[time:time]
                    index_ltp = row1['Close'].values[0]
                    p1['index_ltp'] = index_ltp
                    
                    # finding the call and put strike price
                    atm_strike = get_atm(index_ltp)
                    call_strike = 'BANKNIFTY'+str(atm_strike)+'CE'
                    put_strike = 'BANKNIFTY'+str(atm_strike)+'PE'
                    
                    p1['call_strike'] = call_strike
                    p1['put_strike'] = put_strike
                    
                    # finding the atm_call and atm_put ltp
                    call_ltp = week[(week['Ticker']==call_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    put_ltp = week[(week['Ticker']==put_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    
#                     call_data = call_df.loc[time:time]
#                     put_data = put_df.loc[time:time]
                    
#                     call_ltp = call_data['Close'].values[0]
#                     put_ltp = put_data['Close'].values[0]
                    
                    p1['entry_ce_ltp'] = call_ltp
                    p1['entry_pe_ltp'] = put_ltp
                    
                    # call and put stop loss
                    call_sl = call_ltp*1.17
                    put_sl = put_ltp*1.17
                    p1['call_sl'] = call_sl
                    p1['put_sl'] = put_sl
                    
                if p1['status']==True: 
                    # finding the atm_call and atm_put ltp
                    call_ltp = week[(week['Ticker']==call_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    put_ltp = week[(week['Ticker']==put_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    
                    if call_ltp>=p1['call_sl'] or put_ltp>=p1['put_sl'] or time>=end_time:
                        p1['status'] = False
                        
                        p1['exit_time_call'] = time
                        p1['exit_time_put'] = time
                        
                        p1['exit_ce_ltp'] = call_ltp
                        p1['exit_pe_ltp'] = put_ltp
                        
                        p1['call_pnl'] = p1['entry_ce_ltp']-p1['exit_ce_ltp']
                        p1['put_pnl'] = p1['entry_pe_ltp'] - p1['exit_pe_ltp']
                        
                        p1['day_pnl'] =  p1['call_pnl'] + p1['put_pnl']
                        
                        
                        result.append(p1,ignore_index=True)
                        result.append(p2,ignore_index=True)
                        result.append(p3,ignore_index=True)
                        result.append(p4,ignore_index=True)
                        
                        
                        
                # checking for second position of a day
                if p2['status']==False or time == str(date)[:10]+ ' 10:45:00':
#                     print('check')
                    p2['status'] = True
                    p2['entry_time'] = time
                    
                    # finding the index ltp
                    row1 = bn_index.loc[time:time]
                    index_ltp = row1['Close'].values[0]
                    p2['index_ltp'] = index_ltp
                    
                    # finding the call and put strike price
                    atm_strike = get_atm(index_ltp)
                    call_strike = 'BANKNIFTY'+str(atm_strike)+'CE'
                    put_strike = 'BANKNIFTY'+str(atm_strike)+'PE'
                    
                    p2['call_strike'] = call_strike
                    p2['put_strike'] = put_strike
                    
                    # finding the atm_call and atm_put ltp
                    call_ltp = week[(week['Ticker']==call_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    put_ltp = week[(week['Ticker']==put_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    
                    p2['entry_ce_ltp'] = call_ltp
                    p2['entry_pe_ltp'] = put_ltp
                    
                    # call and put stop loss
                    call_sl = call_ltp*1.17
                    put_sl = put_ltp*1.17
                    p2['call_sl'] = call_sl
                    p2['put_sl'] = put_sl
                    
                if p2['status']==True: 
                    # finding the atm_call and atm_put ltp
                    call_ltp = week[(week['Ticker']==call_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    put_ltp = week[(week['Ticker']==put_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    
                    if call_ltp>=p2['call_sl'] or put_ltp>=p2['put_sl'] or time>=end_time:
                        p2['status'] = False
                        
                        p2['exit_time_call'] = time
                        p2['exit_time_put'] = time
                        
                        p2['exit_ce_ltp'] = call_ltp
                        p2['exit_pe_ltp'] = put_ltp
                        
                        p2['call_pnl'] = p2['entry_ce_ltp']-p2['exit_ce_ltp']
                        p2['put_pnl'] = p2['entry_pe_ltp'] - p2['exit_pe_ltp']
                        
                        p2['day_pnl'] =  p2['call_pnl'] + p2['put_pnl']
                        
            
                        
                # checking for third position of a day
                if p3['status']==False or time == str(date)[:10]+ ' 11:14:00':
#                     print('check')
                    p3['status'] = True
                    p3['entry_time'] = time
                    
                    # finding the index ltp
                    row1 = bn_index.loc[time:time]
                    index_ltp = row1['Close'].values[0]
                    p3['index_ltp'] = index_ltp
                    
                    # finding the call and put strike price
                    atm_strike = get_atm(index_ltp)
                    call_strike = 'BANKNIFTY'+str(atm_strike)+'CE'
                    put_strike = 'BANKNIFTY'+str(atm_strike)+'PE'
                    
                    p3['call_strike'] = call_strike
                    p3['put_strike'] = put_strike
                    
                    # finding the atm_call and atm_put ltp
                    call_ltp = week[(week['Ticker']==call_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    put_ltp = week[(week['Ticker']==put_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    
                    p3['entry_ce_ltp'] = call_ltp
                    p3['entry_pe_ltp'] = put_ltp
                    
                    # call and put stop loss
                    call_sl = call_ltp*1.19
                    put_sl = put_ltp*1.19
                    p3['call_sl'] = call_sl
                    p3['put_sl'] = put_sl
                    
                if p3['status']==True: 
                    # finding the atm_call and atm_put ltp
                    call_ltp = week[(week['Ticker']==call_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    put_ltp = week[(week['Ticker']==put_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    
                    if call_ltp>=p3['call_sl'] or put_ltp>=p3['put_sl'] or time>=end_time:
                        p3['status'] = False
                        
                        p3['exit_time_call'] = time
                        p3['exit_time_put'] = time
                        
                        p3['exit_ce_ltp'] = call_ltp
                        p3['exit_pe_ltp'] = put_ltp
                        
                        p3['call_pnl'] = p3['entry_ce_ltp']-p3['exit_ce_ltp']
                        p3['put_pnl'] = p3['entry_pe_ltp'] - p3['exit_pe_ltp']
                        
                        p3['day_pnl'] =  p3['call_pnl'] + p3['put_pnl']                        

                # checking for fourth position of a day
                if p4['status']==False or time == str(date)[:10]+ ' 11:49:00':
#                     print('check')
                    p4['status'] = True
                    p4['entry_time'] = time
                    
                    # finding the index ltp
                    row1 = bn_index.loc[time:time]
                    index_ltp = row1['Close'].values[0]
                    p4['index_ltp'] = index_ltp
                    
                    # finding the call and put strike price
                    atm_strike = get_atm(index_ltp)
                    call_strike = 'BANKNIFTY'+str(atm_strike)+'CE'
                    put_strike = 'BANKNIFTY'+str(atm_strike)+'PE'
                    
                    p4['call_strike'] = call_strike
                    p4['put_strike'] = put_strike
                    
                    # finding the atm_call and atm_put ltp
                    call_ltp = week[(week['Ticker']==call_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    put_ltp = week[(week['Ticker']==put_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    
                    p4['entry_ce_ltp'] = call_ltp
                    p4['entry_pe_ltp'] = put_ltp
                    
                    # call and put stop loss
                    call_sl = call_ltp*1.23
                    put_sl = put_ltp*1.23
                    p4['call_sl'] = call_sl
                    p4['put_sl'] = put_sl
                    
                if p4['status']==True: 
                    # finding the atm_call and atm_put ltp
                    call_ltp = week[(week['Ticker']==call_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    put_ltp = week[(week['Ticker']==put_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    
                    if call_ltp>=p4['call_sl'] or put_ltp>=p4['put_sl'] or time>=end_time:
                        p4['status'] = False
                        
                        p4['exit_time_call'] = time
                        p4['exit_time_put'] = time
                        
                        p4['exit_ce_ltp'] = call_ltp
                        p4['exit_pe_ltp'] = put_ltp
                        
                        p4['call_pnl'] = p4['entry_ce_ltp']-p4['exit_ce_ltp']
                        p4['put_pnl'] = p4['entry_pe_ltp'] - p4['exit_pe_ltp']
                        
                        p4['day_pnl'] =  p4['call_pnl'] + p4['put_pnl']
                
        elif find_day(date)=='Monday' or find_day(date)=='Friday':
#             print("kfdjkdj")
            for time in intraday_time:
        
                # checking for first position of a day
                if p1['status']==False or time == str(date)[:10]+ ' 09:40:00':
#                     print('check')
                    p1['status'] = True
                    p1['entry_time'] = time
                    
                    # finding the index ltp
                    row1 = bn_index.loc[time:time]
                    index_ltp = row1['Close'].values[0]
                    p1['index_ltp'] = index_ltp
                    
                    # finding the call and put strike price
                    atm_strike = get_atm(index_ltp)
                    call_strike = 'BANKNIFTY'+str(atm_strike)+'CE'
                    put_strike = 'BANKNIFTY'+str(atm_strike)+'PE'
                    
                    p1['call_strike'] = call_strike
                    p1['put_strike'] = put_strike
                    
                    # finding the atm_call and atm_put ltp
                    call_ltp = week[(week['Ticker']==call_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    put_ltp = week[(week['Ticker']==put_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    
#                     call_data = call_df.loc[time:time]
#                     put_data = put_df.loc[time:time]
                    
#                     call_ltp = call_data['Close'].values[0]
#                     put_ltp = put_data['Close'].values[0]
                    
                    p1['entry_ce_ltp'] = call_ltp
                    p1['entry_pe_ltp'] = put_ltp
                    
                    # call and put stop loss
                    call_sl = call_ltp*1.17
                    put_sl = put_ltp*1.17
                    p1['call_sl'] = call_sl
                    p1['put_sl'] = put_sl
                    
                if p1['status']==True: 
                    # finding the atm_call and atm_put ltp
                    call_ltp = week[(week['Ticker']==call_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    put_ltp = week[(week['Ticker']==put_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    
                    if call_ltp>=p1['call_sl'] or put_ltp>=p1['put_sl'] or time>=end_time:
                        p1['status'] = False
                        
                        p1['exit_time_call'] = time
                        p1['exit_time_put'] = time
                        
                        p1['exit_ce_ltp'] = call_ltp
                        p1['exit_pe_ltp'] = put_ltp
                        
                        p1['call_pnl'] = p1['entry_ce_ltp']-p1['exit_ce_ltp']
                        p1['put_pnl'] = p1['entry_pe_ltp'] - p1['exit_pe_ltp']
                        
                        p1['day_pnl'] =  p1['call_pnl'] + p1['put_pnl']
                        
                        
                        result.append(p1,ignore_index=True)
                        result.append(p2,ignore_index=True)
                        result.append(p3,ignore_index=True)
                        result.append(p4,ignore_index=True)
                        
                        
                        
                # checking for second position of a day
                if p2['status']==False or time == str(date)[:10]+ ' 10:45:00':
#                     print('check')
                    p2['status'] = True
                    p2['entry_time'] = time
                    
                    # finding the index ltp
                    row1 = bn_index.loc[time:time]
                    index_ltp = row1['Close'].values[0]
                    p2['index_ltp'] = index_ltp
                    
                    # finding the call and put strike price
                    atm_strike = get_atm(index_ltp)
                    call_strike = 'BANKNIFTY'+str(atm_strike)+'CE'
                    put_strike = 'BANKNIFTY'+str(atm_strike)+'PE'
                    
                    p2['call_strike'] = call_strike
                    p2['put_strike'] = put_strike
                    
                    # finding the atm_call and atm_put ltp
                    call_ltp = week[(week['Ticker']==call_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    put_ltp = week[(week['Ticker']==put_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    
                    p2['entry_ce_ltp'] = call_ltp
                    p2['entry_pe_ltp'] = put_ltp
                    
                    # call and put stop loss
                    call_sl = call_ltp*1.17
                    put_sl = put_ltp*1.17
                    p2['call_sl'] = call_sl
                    p2['put_sl'] = put_sl
                    
                if p2['status']==True: 
                    # finding the atm_call and atm_put ltp
                    call_ltp = week[(week['Ticker']==call_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    put_ltp = week[(week['Ticker']==put_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    
                    if call_ltp>=p2['call_sl'] or put_ltp>=p2['put_sl'] or time>=end_time:
                        p2['status'] = False
                        
                        p2['exit_time_call'] = time
                        p2['exit_time_put'] = time
                        
                        p2['exit_ce_ltp'] = call_ltp
                        p2['exit_pe_ltp'] = put_ltp
                        
                        p2['call_pnl'] = p2['entry_ce_ltp']-p2['exit_ce_ltp']
                        p2['put_pnl'] = p2['entry_pe_ltp'] - p2['exit_pe_ltp']
                        
                        p2['day_pnl'] =  p2['call_pnl'] + p2['put_pnl']
                        
            
                        
                # checking for third position of a day
                if p3['status']==False or time == str(date)[:10]+ ' 11:14:00':
#                     print('check')
                    p3['status'] = True
                    p3['entry_time'] = time
                    
                    # finding the index ltp
                    row1 = bn_index.loc[time:time]
                    index_ltp = row1['Close'].values[0]
                    p3['index_ltp'] = index_ltp
                    
                    # finding the call and put strike price
                    atm_strike = get_atm(index_ltp)
                    call_strike = 'BANKNIFTY'+str(atm_strike)+'CE'
                    put_strike = 'BANKNIFTY'+str(atm_strike)+'PE'
                    
                    p3['call_strike'] = call_strike
                    p3['put_strike'] = put_strike
                    
                    # finding the atm_call and atm_put ltp
                    call_ltp = week[(week['Ticker']==call_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    put_ltp = week[(week['Ticker']==put_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    
                    p3['entry_ce_ltp'] = call_ltp
                    p3['entry_pe_ltp'] = put_ltp
                    
                    # call and put stop loss
                    call_sl = call_ltp*1.19
                    put_sl = put_ltp*1.19
                    p3['call_sl'] = call_sl
                    p3['put_sl'] = put_sl
                    
                if p3['status']==True: 
                    # finding the atm_call and atm_put ltp
                    call_ltp = week[(week['Ticker']==call_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    put_ltp = week[(week['Ticker']==put_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    
                    if call_ltp>=p3['call_sl'] or put_ltp>=p3['put_sl'] or time>=end_time:
                        p3['status'] = False
                        
                        p3['exit_time_call'] = time
                        p3['exit_time_put'] = time
                        
                        p3['exit_ce_ltp'] = call_ltp
                        p3['exit_pe_ltp'] = put_ltp
                        
                        p3['call_pnl'] = p3['entry_ce_ltp']-p3['exit_ce_ltp']
                        p3['put_pnl'] = p3['entry_pe_ltp'] - p3['exit_pe_ltp']
                        
                        p3['day_pnl'] =  p3['call_pnl'] + p3['put_pnl']                        

                # checking for fourth position of a day
                if p4['status']==False or time == str(date)[:10]+ ' 11:49:00':
#                     print('check')
                    p4['status'] = True
                    p4['entry_time'] = time
                    
                    # finding the index ltp
                    row1 = bn_index.loc[time:time]
                    index_ltp = row1['Close'].values[0]
                    p4['index_ltp'] = index_ltp
                    
                    # finding the call and put strike price
                    atm_strike = get_atm(index_ltp)
                    call_strike = 'BANKNIFTY'+str(atm_strike)+'CE'
                    put_strike = 'BANKNIFTY'+str(atm_strike)+'PE'
                    
                    p4['call_strike'] = call_strike
                    p4['put_strike'] = put_strike
                    
                    # finding the atm_call and atm_put ltp
                    call_ltp = week[(week['Ticker']==call_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    put_ltp = week[(week['Ticker']==put_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    
                    p4['entry_ce_ltp'] = call_ltp
                    p4['entry_pe_ltp'] = put_ltp
                    
                    # call and put stop loss
                    call_sl = call_ltp*1.23
                    put_sl = put_ltp*1.23
                    p4['call_sl'] = call_sl
                    p4['put_sl'] = put_sl
                    
                if p4['status']==True: 
                    # finding the atm_call and atm_put ltp
                    call_ltp = week[(week['Ticker']==call_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    put_ltp = week[(week['Ticker']==put_strike) & (week['Date/Time']==time)]['Close'].values[0]
                    
                    if call_ltp>=p4['call_sl'] or put_ltp>=p4['put_sl'] or time>=end_time:
                        p4['status'] = False
                        
                        p4['exit_time_call'] = time
                        p4['exit_time_put'] = time
                        
                        p4['exit_ce_ltp'] = call_ltp
                        p4['exit_pe_ltp'] = put_ltp
                        
                        p4['call_pnl'] = p4['entry_ce_ltp']-p4['exit_ce_ltp']
                        p4['put_pnl'] = p4['entry_pe_ltp'] - p4['exit_pe_ltp']
                        
                        p4['day_pnl'] =  p4['call_pnl'] + p4['put_pnl']
                


        result = result.append(p1,ignore_index=True)
        result = result.append(p2,ignore_index=True)
        result = result.append(p3,ignore_index=True)
        result = result.append(p4,ignore_index=True)


In [54]:
unique_day

[datetime.datetime(2022, 1, 14, 0, 0),
 datetime.datetime(2022, 1, 17, 0, 0),
 datetime.datetime(2022, 1, 18, 0, 0),
 datetime.datetime(2022, 1, 19, 0, 0),
 datetime.datetime(2022, 1, 20, 0, 0)]

In [43]:
p1

{'status': False,
 'entry_time': '2022-01-14 15:05:00',
 'index_ltp': 38424.0,
 'call_strike': 'BANKNIFTY38400CE',
 'put_strike': 'BANKNIFTY38400PE',
 'entry_ce_ltp': 438.6,
 'entry_pe_ltp': 374.25,
 'call_sl': 513.162,
 'put_sl': 437.87249999999995,
 'exit_time_call': '2022-01-14 15:20:00',
 'exit_time_put': '2022-01-14 15:20:00',
 'exit_ce_ltp': 405.5,
 'exit_pe_ltp': 419.15,
 'call_pnl': 33.10000000000002,
 'put_pnl': -44.89999999999998,
 'day_pnl': -11.799999999999955,
 'cumulative_pnl': 0,
 'remark': None}

In [38]:
p1

{'status': False,
 'entry_time': '2022-01-14 15:05:00',
 'index_ltp': 38424.0,
 'call_strike': 'BANKNIFTY38400CE',
 'put_strike': 'BANKNIFTY38400PE',
 'entry_ce_ltp': 438.6,
 'entry_pe_ltp': 374.25,
 'call_sl': 513.162,
 'put_sl': 437.87249999999995,
 'exit_time_call': '2022-01-14 15:20:00',
 'exit_time_put': '2022-01-14 15:20:00',
 'exit_ce_ltp': 405.5,
 'exit_pe_ltp': 419.15,
 'call_pnl': 33.10000000000002,
 'put_pnl': -44.89999999999998,
 'day_pnl': -11.799999999999955,
 'cumulative_pnl': 0,
 'remark': None}

In [39]:
p2

{'status': False,
 'entry_time': '2022-01-14 10:45:00',
 'index_ltp': 38134.1,
 'call_strike': 'BANKNIFTY38100CE',
 'put_strike': 'BANKNIFTY38100PE',
 'entry_ce_ltp': 467.8,
 'entry_pe_ltp': 390.1,
 'call_sl': 547.326,
 'put_sl': 456.417,
 'exit_time_call': '2022-01-14 15:20:00',
 'exit_time_put': '2022-01-14 15:20:00',
 'exit_ce_ltp': 405.5,
 'exit_pe_ltp': 419.15,
 'call_pnl': 62.30000000000001,
 'put_pnl': -29.049999999999955,
 'day_pnl': 33.25000000000006,
 'cumulative_pnl': 0,
 'remark': None}

In [40]:
p3

{'status': False,
 'entry_time': '2022-01-14 15:05:00',
 'index_ltp': 38424.0,
 'call_strike': 'BANKNIFTY38400CE',
 'put_strike': 'BANKNIFTY38400PE',
 'entry_ce_ltp': 438.6,
 'entry_pe_ltp': 374.25,
 'call_sl': 521.934,
 'put_sl': 445.35749999999996,
 'exit_time_call': '2022-01-14 15:20:00',
 'exit_time_put': '2022-01-14 15:20:00',
 'exit_ce_ltp': 405.5,
 'exit_pe_ltp': 419.15,
 'call_pnl': 33.10000000000002,
 'put_pnl': -44.89999999999998,
 'day_pnl': -11.799999999999955,
 'cumulative_pnl': 0,
 'remark': None}

In [41]:
p4

{'status': False,
 'entry_time': '2022-01-14 11:49:00',
 'index_ltp': 38117.4,
 'call_strike': 'BANKNIFTY38100CE',
 'put_strike': 'BANKNIFTY38100PE',
 'entry_ce_ltp': 439.15,
 'entry_pe_ltp': 391.5,
 'call_sl': 540.1545,
 'put_sl': 481.545,
 'exit_time_call': '2022-01-14 15:20:00',
 'exit_time_put': '2022-01-14 15:20:00',
 'exit_ce_ltp': 405.5,
 'exit_pe_ltp': 419.15,
 'call_pnl': 33.64999999999998,
 'put_pnl': -27.649999999999977,
 'day_pnl': 6.0,
 'cumulative_pnl': 0,
 'remark': None}

In [42]:
p1

{'status': False,
 'entry_time': '2022-01-14 15:05:00',
 'index_ltp': 38424.0,
 'call_strike': 'BANKNIFTY38400CE',
 'put_strike': 'BANKNIFTY38400PE',
 'entry_ce_ltp': 438.6,
 'entry_pe_ltp': 374.25,
 'call_sl': 513.162,
 'put_sl': 437.87249999999995,
 'exit_time_call': '2022-01-14 15:20:00',
 'exit_time_put': '2022-01-14 15:20:00',
 'exit_ce_ltp': 405.5,
 'exit_pe_ltp': 419.15,
 'call_pnl': 33.10000000000002,
 'put_pnl': -44.89999999999998,
 'day_pnl': -11.799999999999955,
 'cumulative_pnl': 0,
 'remark': None}

In [47]:
result = result.append(p1,ignore_index=True)

In [48]:
result.append(p2,ignore_index=True)

,call_pnl,call_sl,call_strike,cumulative_pnl,day_pnl,entry_ce_ltp,entry_pe_ltp,entry_time,exit_ce_ltp,exit_pe_ltp,exit_time_call,exit_time_put,index_ltp,put_pnl,put_sl,put_strike,remark,status
0,33.1,513.162,BANKNIFTY38400CE,0.0,-11.80,438.6,374.25,2022-01-14 15:05:00,405.5,419.15,2022-01-14 15:20:00,2022-01-14 15:20:00,38424.0,-44.90,437.8725,BANKNIFTY38400PE,None,0.0
1,62.3,547.326,BANKNIFTY38100CE,0.0,33.25,467.8,390.10,2022-01-14 10:45:00,405.5,419.15,2022-01-14 15:20:00,2022-01-14 15:20:00,38134.1,-29.05,456.4170,BANKNIFTY38100PE,None,0.0


In [55]:
result

,call_pnl,call_sl,call_strike,cumulative_pnl,day_pnl,entry_ce_ltp,entry_pe_ltp,entry_time,exit_ce_ltp,exit_pe_ltp,exit_time_call,exit_time_put,index_ltp,put_pnl,put_sl,put_strike,remark,status
0,59.30,500.8770,BANKNIFTY35500CE,0.0,55.45,428.10,371.15,2021-12-31 09:40:00,368.80,375.00,2021-12-31 15:20:00,2021-12-31 15:20:00,35543.3,-3.85,434.2455,BANKNIFTY35500PE,None,0.0
1,50.25,490.2885,BANKNIFTY35400CE,0.0,19.10,419.05,343.85,2021-12-31 10:45:00,368.80,375.00,2021-12-31 15:20:00,2021-12-31 15:20:00,35445.8,-31.15,402.3045,BANKNIFTY35400PE,None,0.0
2,23.60,466.9560,BANKNIFTY35500CE,0.0,15.90,392.40,367.30,2021-12-31 11:14:00,368.80,375.00,2021-12-31 15:20:00,2021-12-31 15:20:00,35496.7,-7.70,437.0870,BANKNIFTY35500PE,None,0.0
3,22.20,480.9300,BANKNIFTY35500CE,0.0,7.30,391.00,360.10,2021-12-31 11:49:00,368.80,375.00,2021-12-31 15:20:00,2021-12-31 15:20:00,35513.4,-14.90,442.9230,BANKNIFTY35500PE,None,0.0
4,7.50,413.3610,BANKNIFTY36400CE,0.0,33.30,353.30,303.00,2022-01-03 14:34:00,345.80,277.20,2022-01-03 15:20:00,2022-01-03 15:20:00,36432.9,25.80,354.5100,BANKNIFTY36400PE,None,0.0
5,7.50,413.3610,BANKNIFTY36400CE,0.0,33.30,353.30,303.00,2022-01-03 14:34:00,345.80,277.20,2022-01-03 15:20:00,2022-01-03 15:20:00,36432.9,25.80,354.5100,BANKNIFTY36400PE,None,0.0
6,3.50,415.6670,BANKNIFTY36300CE,0.0,16.00,349.30,289.70,2022-01-03 14:19:00,345.80,277.20,2022-01-03 15:20:00,2022-01-03 15:20:00,36337.9,12.50,344.7430,BANKNIFTY36300PE,None,0.0
7,3.50,429.6390,BANKNIFTY36300CE,0.0,16.00,349.30,289.70,2022-01-03 14:19:00,345.80,277.20,2022-01-03 15:20:00,2022-01-03 15:20:00,36337.9,12.50,356.3310,BANKNIFTY36300PE,None,0.0
8,0.00,NaN,None,0.0,0.00,NaN,NaN,None,NaN,NaN,None,None,NaN,0.00,NaN,None,None,0.0
9,0.00,NaN,None,0.0,0.00,NaN,NaN,None,NaN,NaN,None,None,NaN,0.00,NaN,None,None,0.0
